In [ ]:
import sys

sys.path.insert(0, '..')

from functions.adjust_cases_functions import prepare_cases 
from functions.general_utils import  get_bool
from models.seird_model import SEIRD

from statsmodels.tsa.arima.model import ARIMA


import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import numpy as np
import os

from global_config import config


poly_run  = 11001
name_dir  = 'bogota'
drop_last_weeks = False

data_dir            = config.get_property('data_dir_covid')
geo_dir             = config.get_property('geo_dir')
data_dir_mnps       = config.get_property('data_dir_col')
results_dir         = config.get_property('results_dir')

agglomerated_folder = os.path.join(data_dir, 'data_stages', 'colombia', 'agglomerated', 'geometry' )
raw_folder          = os.path.join(data_dir, 'data_stages', 'colombia', 'raw', 'cases' )

polygons = pd.read_csv(os.path.join(agglomerated_folder, 'polygons.csv')).set_index('poly_id')
polygons = polygons.loc[poly_run]


data_raw  =  pd.read_csv(os.path.join(raw_folder, 'cases_raw.csv'), dayfirst=True)

data  =  pd.read_csv(os.path.join(agglomerated_folder, 'cases.csv'), parse_dates=['date_time'], dayfirst=True).set_index('poly_id')
data  = data.loc[poly_run].set_index('date_time')
data  = data.resample('D').sum().fillna(0)[['num_cases','num_diseased']]
data  = prepare_cases(data, col='num_cases', cutoff=0)   
data  = prepare_cases(data, col='num_diseased', cutoff=0)



In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15.5, 14.2))

ax[0].plot(data.index.values, data.smoothed_num_cases, color='k', linewidth=2)
ax[0].scatter(data.index.values, data.num_cases, edgecolor='k', facecolor='w')

ax[1].plot(data.index.values, data.smoothed_num_diseased, color='r', linewidth=2)
ax[1].scatter(data.index.values, data.num_diseased, edgecolor='r', facecolor='w')




In [1]:
import argparse
import logging
import math
import re
from collections import OrderedDict

import torch

import pyro
import pyro.distributions as dist
import pyro.distributions.hmm
import pyro.poutine as poutine
from pyro.infer import MCMC, NUTS, config_enumerate, infer_discrete
from pyro.infer.autoguide import init_to_value
from pyro.ops.special import safe_log
from pyro.ops.tensor_utils import convolve
from pyro.util import warn_if_nan

logging.basicConfig(format='%(message)s', level=logging.INFO)



In [2]:
import argparse
import logging

import torch

import pyro
import pyro.distributions as dist
from pyro.infer import SMCFilter

logging.basicConfig(format="%(relativeCreated) 9d %(message)s", level=logging.INFO)


class SimpleHarmonicModel:

    def __init__(self, process_noise, measurement_noise):
        self.A = torch.tensor([[0., 1.],
                               [-1., 0.]])
        self.B = torch.tensor([3., 3.])
        self.sigma_z = torch.tensor(process_noise)
        self.sigma_y = torch.tensor(measurement_noise)

    def init(self, state, initial):
        self.t = 0
        state["z"] = pyro.sample("z_init", dist.Delta(initial, event_dim=1))

    def step(self, state, y=None):
        self.t += 1
        state["z"] = pyro.sample(
            "z_{}".format(self.t),
            dist.Normal(state["z"].matmul(self.A), self.B*self.sigma_z).to_event(1))
    
        y = pyro.sample("y_{}".format(self.t),
                        dist.Normal(state["z"][..., 0], self.sigma_y),
                        obs=y)
        return state["z"], y


class SimpleHarmonicModel_Guide:

    def __init__(self, model):
        self.model = model

    def init(self, state, initial):
        self.t = 0
        pyro.sample("z_init", dist.Delta(initial, event_dim=1))

    def step(self, state, y=None):
        self.t += 1

        # Proposal distribution
        pyro.sample(
            "z_{}".format(self.t),
            dist.Normal(state["z"].matmul(self.model.A), torch.tensor([1., 1.])).to_event(1))


def generate_data(args):
    model = SimpleHarmonicModel(args.process_noise, args.measurement_noise)

    state = {}
    initial = torch.tensor([1., 0.])
    model.init(state, initial=initial)
    zs = [initial]
    ys = [None]
    for t in range(args.num_timesteps):
        z, y = model.step(state)
        zs.append(z)
        ys.append(y)

    return zs, ys

In [3]:
parser = argparse.ArgumentParser(description="Simple Harmonic Oscillator w/ SMC Filtering Inference")
parser.add_argument("-n", "--num-timesteps", default=500, type=int)
parser.add_argument("-p", "--num-particles", default=100, type=int)
parser.add_argument("--process-noise", default=1., type=float)
parser.add_argument("--measurement-noise", default=1., type=float)
parser.add_argument("--seed", default=0, type=int)

args = parser.parse_args([])

In [4]:
pyro.set_rng_seed(args.seed)

model = SimpleHarmonicModel(args.process_noise, args.measurement_noise)
guide = SimpleHarmonicModel_Guide(model)

smc = SMCFilter(model, guide, num_particles=args.num_particles, max_plate_nesting=0)

logging.info("Generating data")
zs, ys = generate_data(args)

logging.info("Filtering")

smc.init(initial=torch.tensor([1., 0.]))
for y in ys[1:]:
    smc.step(y)

logging.info("At final time step:")
z = smc.get_empirical()["z"]

Generating data
Filtering
At final time step:


In [16]:
zs[-1], z.mean, z.variance ** 0.5

(tensor([ 0.0809, 67.0811]),
 tensor([-1.1345, 65.9661]),
 tensor([0.8514, 1.6066]))